##### Copyright 2024 Google LLC.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [2]:
# The non-source code materials on this page are licensed under Creative Commons - Attribution-ShareAlike CC-BY-SA 4.0,
# https://creativecommons.org/licenses/by-sa/4.0/legalcode.

# Search re-ranking using Gemini embeddings

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/docs/search_reranking_using_embeddings"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />View on Google AI</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/docs/search_reranking_using_embeddings.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google/generative-ai-docs/blob/main/site/en/docs/search_reranking_using_embeddings.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

This notebook demonstrates the use of embeddings to re-rank search results. This walkthrough will focus on the following objectives:



1.   Setting up your development environment and API access to use Gemini.
2.   Using Gemini's function calling support to access the Wikipedia API.
3.   Embedding content via Gemini API.
4.   Re-ranking the search results.


This is how you will implement search re-ranking:


1.   User will query the model.
2.   You will use Wikipedia API to return relevant search results.
3.   The search results will be embedded and their relevance will be evaluated by calculating distance metrics like cosine similarity, dot product, etc.
4.   Most relevant result will be returned as the final answer.

## Prerequisites

You can run this quickstart in [Google Colab](https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/docs/search_reranking_using_embeddings.ipynb), which runs this notebook directly in the browser and does not require additional environment configuration.


## Setup


The Python SDK for the Gemini API, is contained in the [`google-generativeai`](https://pypi.org/project/google-generativeai/) package. You will also need to install the **Wikipedia** API.


In [ ]:
!pip install -q google-generativeai

In [ ]:
!pip install -q wikipedia

> Note: This library was designed for ease of use and simplicity, not for advanced use. If you plan on doing serious scraping or automated requests, please use [Pywikipediabot](http://www.mediawiki.org/wiki/Manual:Pywikipediabot) or one of the other more advanced [Python MediaWiki API wrappers](http://en.wikipedia.org/wiki/Wikipedia:Creating_a_bot#Python), which has a larger API, rate limiting, and other features so you can be considerate of the MediaWiki infrastructure.

Import the necessary packages.

In [ ]:
import google.generativeai as genai
import google.ai.generativelanguage as glm

import wikipedia
from wikipedia.exceptions import DisambiguationError, PageError

from google.colab import userdata

import numpy as np

### Grab an API key

Before you can use the Gemini API, you must first obtain an API key. If you don't already have one, create a key with one click in Google AI Studio.

<a class="button button-primary" href="https://makersuite.google.com/app/apikey" target="_blank" rel="noopener noreferrer">Get an API key</a>


In Colab, add the key to the secrets manager under the "🔑" in the left panel. Give it the name `GOOGLE_API_KEY`.

Once you have the API key, pass it to the SDK. You can do this in two ways:

* Put the key in the `GOOGLE_API_KEY` environment variable (the SDK will automatically pick it up from there).
* Pass the key to `genai.configure(api_key=...)`


In [ ]:
# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

Key Point: Next, you will choose a model. Any embedding model will work for this tutorial, but for real applications it's important to choose a specific model and stick with it. The outputs of different models are not compatible with each other.

**Note**: At this time, the Gemini API is [only available in certain regions](https://developers.generativeai.google/available_regions).

## Define tools

As stated earlier, this tutorial uses Gemini's function calling support to access the Wikipedia API. Please refer to the [docs](https://ai.google.dev/docs/function_calling) to learn more about function calling.

Currently, the Gemini Python SDK requires you to wrap your tool definition in `glm.FunctionDeclaration` object.

To cater to the search engine needs, you will design this function in the following way:


*   For each search query, the search engine will use the `wikipedia.search` method to get relevant topics.
*   From the relevant topics, the engine will choose `n_topics(int)` top candidates and will use `gemini-pro` to extract relevant information from the page.
*   The engine will avoid duplicate entries by maintaining a search history.


In [ ]:
def wikipedia_search(search_queries: list[str], n_topics=3) -> list[str]:
  """Given a query by the user, this function generates search queries which upon
  seaching Wikipedia might answer the user's quesition."""
  search_history = set() # tracking search history
  search_urls = []
  mining_model = genai.GenerativeModel('gemini-pro')
  summary_results = []

  for query in search_queries:
    print(f'Searching for "{query}"')
    search_terms = wikipedia.search(query)
    print(f"Related search terms: {search_terms[:n_topics]}")
    for search_term in search_terms[:n_topics]: # select first `n_topics` candidates
      if search_term in search_history: # check if the topic is already covered
        continue
      print(f'Fetching page: "{search_term}"')
      search_history.add(search_term) # add to search history
      try:
        # extract the relevant data by using `gemini-pro` model
        page = wikipedia.page(search_term, auto_suggest=False)
        print(f"Information Source: {page.url}")
        search_urls.append(page.url)
        page = page.content
        page = page.replace('\n', ' ')
        response = mining_model.generate_content("""Extract relevant information
        about user's query: {query} from this source: {source}.
        Note: Do not summarize. Only Extract and return the relevant information
        """.format(query=query, source=page)
        )
        summary_results.append(response.text)

      except DisambiguationError:
        print(f"""Results when searching for "{search_term}" (originally for "{query}")
        were ambiguous, hence skipping""")

      except PageError:
        print(f'{search_term} did not match with any page id, hence skipping.')

  print(f"Information Sources: {search_urls}")

  return summary_results


In [ ]:
example = wikipedia_search(["What are LLMs?"])

### Signature

You will start by defining the function's signature. To do this, create `glm.Tool` which contains the function declaration. Here, `Tool` is code that allows the model to interact with external systems to perform an action(s), outside of knowledge and scope of the model. A `FunctionDeclaration` represents a block of code that can be used as a `Tool` by the model and executed by the client. One of the parameters of `FunctionDeclaration` is `Schema`, which allows the definition of input and output data types.

In [ ]:
tools = glm.Tool(
    function_declarations=[
        glm.FunctionDeclaration(
            name=wikipedia_search.__name__,
            description=wikipedia_search.__doc__,
            parameters=glm.Schema(
                type=glm.Type.OBJECT,
                properties={
                    "search_queries": glm.Schema(type=glm.Type.ARRAY,
                            description="""List of search queries which might
                            answer user's question"""
                            )
                },
                required=["search_queries"]
            ),
        )
    ]
  )

## Generate supporting search queries

In order to have multiple supporting search queries to the user's original query, you will ask the model to generate more such queries. This would help the engine to cover the asked question on comprehensive levels.

In [ ]:
instructions = """You have access to the Wikipedia API which you will be using
to answer a user's query. Your job is to generate a list of search queries which
might answer a user's question. Be creative by using various key-phrases from
the user's query. To generate variety of queries, ask questions which are
related to  the user's query that might help to find the answer. The more
queries you generate the better are the odds of you finding the correct answer.
Here is an example:

query: Tell me about Cricket World cup 2023 winners.

output: function_call(wikipedia_search, args:['What is the name of the team that
won the Cricket World Cup 2023?', 'Who was the captain of the Cricket World Cup
2023 winning team?', 'Which country hosted the Cricket World Cup 2023?', 'What
was the venue of the Cricket World Cup 2023 final match?', 'Cricket World cup 2023',
'Who lifted the Cricket World Cup 2023 trophy?', ...])

query: {query}
"""

In [ ]:
model = genai.GenerativeModel('gemini-pro', tools=[tools], generation_config={'temperature': 0.6})

In order to yield creative and a more random variety of questions, you will set the model's temperature parameter to a value higher. Values can range from [0.0,1.0], inclusive. A value closer to 1.0 will produce responses that are more varied and creative, while a value closer to 0.0 will typically result in more straightforward responses from the model.

> Note: Explore more parameters from [genai.GenerativeModel.GenerationConfig](https://ai.google.dev/api/python/google/generativeai/GenerationConfig) class to control model's response in a better way.

In [ ]:
query = "Explain how deep-sea life survives."

In [ ]:
res = model.generate_content(instructions.format(query=query))
print(res.candidates[0])

Awesome! The model was able to generate multiple supporting queries. Note that the model used provided tools to generate the answer.

Next, you can easily extract the function name and arguments as follows:

In [ ]:
fc = res.candidates[0].content.parts[0].function_call
args = fc.args
name = fc.name

## Execute the Wikipedia API (search engine)

Call the function with generated arguments to get the results.

In [ ]:
summaries = wikipedia_search(args.get('search_queries', []))

## Re-ranking the search results

Helper function to embed the content:

In [ ]:
def get_embeddings(content: list[str]) -> np.ndarray:
  embeddings = genai.embed_content('models/embedding-001', content, 'SEMANTIC_SIMILARITY')
  embds = embeddings.get('embedding', None)
  embds = np.array(embds).reshape(len(embds), -1)
  return embds

Please refer to the [embeddings_guide](https://ai.google.dev/docs/embeddings_guide) for more information on embeddings.

Your next step is to define functions that you can use to calculate similarity scores between two embedding vectors. These scores will help you decide which embedding vector is the most relevant vector to the user's query.


You will now implement **cosine similarity** as your metric. Here returned embedding vectors will be of unit length and hence their L1 norm (`np.linalg.norm()`) will be ~1. Hence, calculating **cosine similarity** is esentially same as calculating their **dot product score**.

In [ ]:
def dot_product(a: np.ndarray, b: np.ndarray):
  return (a @ b.T)

### Similarity with user's query

Now it's time to find the most relevant search result returned by the Wikipedia API.

Use Gemini API to get embeddings for user's query and search results.

In [ ]:
search_res = get_embeddings(summaries)
embedded_query = get_embeddings([query])

Calculate similarity score:

In [ ]:
sim_value = dot_product(search_res, embedded_query)

using `np.argmax` best candidate is selected.

**Users's Input:** Explain how deep-sea life survives.

**Answer:**

In [ ]:
summaries[np.argmax(sim_value)]

### Similarity with Hypothetical Document Embeddings (HyDE)

Drawing inspiration from [[Gao et al](https://arxiv.org/abs/2212.10496)] the objective here is to generate a template answer to the user's query using `gemini-pro`'s internal knowledge. This hypothetical answer will serve as a baseline to calculate relevance of all the search results.

In [ ]:
hypothetical_ans_model = genai.GenerativeModel('gemini-pro')
res = hypothetical_ans_model.generate_content("""Generate a hypothetical answer
to the user's query by using your own knowledge. Assume that you know everything
about the said topic. Do not use factual information, instead use placeholders
to complete your answer. Your answer should feel like it has been written by a human.

query: {query}""".format(query=query))
res.text

Use Gemini API to get embeddings for the baseline answer and compare them with search results

In [ ]:
hypothetical_ans = get_embeddings([res.text])

Calculate similarity scores to rank the search results

In [ ]:
sim_value = dot_product(search_res, hypothetical_ans)

In [ ]:
sim_value

using `np.argmax` best candidate is selected.

**Users's Input:** Explain how deep-sea life survives.

**Answer:**

In [ ]:
summaries[np.argmax(sim_value)]

You have now created a search re-ranking engine using embeddings!

## Next steps

To learn how to use other services in the Gemini API, visit the [Python quickstart](https://ai.google.dev/tutorials/python_quickstart). To learn more about how you can use the embeddings, check out the [examples](../examples?keywords=embed) available.